# Data Cleaning

In [1]:
import numpy as np
import pandas as pd
import re

Let's read in our Food Inspections dataset.

In [2]:
raw_df = pd.read_csv("../Datasets/Food_Inspections.csv")

print(raw_df.head())
print(raw_df.info())

   Inspection ID                                      DBA Name  \
0        1170324  CAPTAIN HOOKS FISH & CHICKEN/FIREHOUSE GRILL   
1        2614750                           CHICK-FIL-A PULLMAN   
2        2609909                                  HAPPY MARKET   
3        2609927                    SAT KAIVAL FOOD INC/SUBWAY   
4        2608378                                   Babas Halal   

                                       AKA Name  License #  Facility Type  \
0  CAPTAIN HOOKS FISH & CHICKEN FIREHOUSE GRILL  1804138.0     Restaurant   
1                           CHICK-FIL-A PULLMAN  3015728.0     Restaurant   
2                                  HAPPY MARKET  2912802.0  Grocery Store   
3                    SAT KAIVAL FOOD INC/SUBWAY  2728400.0     Restaurant   
4                                   Babas Halal  2684170.0     Restaurant   

              Risk               Address     City State      Zip  \
0    Risk 1 (High)      5648 W ROOSEVELT  CHICAGO    IL  60644.0   
1   

In [3]:
# Print the shape of the DataFrame
print("Shape of the DataFrame:", raw_df.shape)

Shape of the DataFrame: (290061, 17)


The column with the greatest number of null values is `Violations` which most likely means that these places passed their annual inspections. However, if only over 80,000 spots out of 290,000 passed, this is bad news for the people of Chicago. We must transform the data further so we can understand how many times each establishment is on here annually. 

In [4]:
# Print the count of missing values in each column
missing_data = raw_df.isnull().sum()
print("Missing data counts:\n", missing_data[missing_data > 0])

Missing data counts:
 AKA Name            2422
License #             18
Facility Type       5208
Risk                  81
City                 159
State                 58
Zip                   39
Inspection Type        1
Violations         80512
Latitude             992
Longitude            992
Location             992
dtype: int64


In [5]:
print(raw_df['Inspection ID'].nunique(), raw_df['License #'].nunique())

290061 46542


In [6]:
# Print the unique value counts for each column
unique_counts = raw_df.nunique()
print("Unique value counts:\n", unique_counts)

Unique value counts:
 Inspection ID      290061
DBA Name            33477
AKA Name            31866
License #           46542
Facility Type         520
Risk                    4
Address             31275
City                   86
State                   6
Zip                   127
Inspection Date      3854
Inspection Type       110
Results                 7
Violations         208120
Latitude            18526
Longitude           18526
Location            18526
dtype: int64


This doesn't provide much because our Inspections data doesn't provide any insights.

In [7]:
# Print descriptive statistics for numerical columns
print("Descriptive statistics for numerical columns:\n", raw_df.describe())

Descriptive statistics for numerical columns:
        Inspection ID     License #            Zip       Latitude  \
count   2.900610e+05  2.900430e+05  290022.000000  289069.000000   
mean    1.797472e+06  1.760551e+06   60628.653523      41.880692   
std     7.293133e+05  9.368281e+05     154.691803       0.081040   
min     4.424700e+04  0.000000e+00   10014.000000      41.644670   
25%     1.335240e+06  1.381404e+06   60614.000000      41.831819   
50%     1.982701e+06  2.073531e+06   60625.000000      41.891797   
75%     2.492873e+06  2.417842e+06   60643.000000      41.939768   
max     2.615858e+06  9.999999e+06   90504.000000      42.021064   

           Longitude  
count  289069.000000  
mean      -87.676366  
std         0.058374  
min       -87.906874  
25%       -87.707649  
50%       -87.666369  
75%       -87.634874  
max       -87.525094  


We should convert  `Zip` and `License #` to strings, and Inspection Date to date time.

In [8]:
# Print the data types of each column
print("Data types of each column:\n", raw_df.dtypes)

Data types of each column:
 Inspection ID        int64
DBA Name            object
AKA Name            object
License #          float64
Facility Type       object
Risk                object
Address             object
City                object
State               object
Zip                float64
Inspection Date     object
Inspection Type     object
Results             object
Violations          object
Latitude           float64
Longitude          float64
Location            object
dtype: object


In [9]:
raw_df['License #'] = raw_df['License #'].astype(str)
raw_df['Zip'] = raw_df['Zip'].astype(str)
raw_df['Facility Type'] = raw_df['Facility Type'].astype(str)
raw_df['Inspection Date'] = pd.to_datetime(raw_df['Inspection Date'])
raw_df = raw_df.rename(columns={'License #': 'License Number'})

In [10]:
print(raw_df.columns)

Index(['Inspection ID', 'DBA Name', 'AKA Name', 'License Number',
       'Facility Type', 'Risk', 'Address', 'City', 'State', 'Zip',
       'Inspection Date', 'Inspection Type', 'Results', 'Violations',
       'Latitude', 'Longitude', 'Location'],
      dtype='object')


Let's dive a little deeper into each column to see if there are any inconsistencies.

In [11]:
print(raw_df['City'].value_counts())

City
CHICAGO         288924
Chicago            453
chicago            150
CCHICAGO            60
SCHAUMBURG          28
                 ...  
ALGONQUIN            1
GRIFFITH             1
NEW YORK             1
NEW HOLSTEIN         1
GRAYSLAKE            1
Name: count, Length: 86, dtype: int64


In [12]:
print(raw_df['Facility Type'].value_counts())

Facility Type
Restaurant                         195790
Grocery Store                       35457
School                              18322
Children's Services Facility         6707
nan                                  5208
                                    ...  
PREPACAKAGED FOODS                      1
COFFEE ROASTER                          1
RESTAURANT AND LIQUOR                   1
HEALTH CENTER/NUTRITION CLASSES         1
TAVERN-LIQUOR                           1
Name: count, Length: 521, dtype: int64


In [13]:
print(raw_df['Risk'].value_counts())

Risk
Risk 1 (High)      214173
Risk 2 (Medium)     52646
Risk 3 (Low)        23090
All                    71
Name: count, dtype: int64


In [14]:
print(raw_df['Results'].value_counts())

Results
Pass                    149658
Fail                     56275
Pass w/ Conditions       43684
Out of Business          24203
No Entry                 12281
Not Ready                 3868
Business Not Located        92
Name: count, dtype: int64


In [15]:
print(raw_df['State'].value_counts())

State
IL    289988
IN        10
CA         2
CO         1
NY         1
WI         1
Name: count, dtype: int64


In [16]:
# Lower and capitalize every word
raw_df['City'] = raw_df['City'].str.lower()
raw_df['City'] = raw_df['City'].str.title()
raw_df['Facility Type'] = raw_df['Facility Type'].str.lower()
raw_df['Facility Type'] = raw_df['Facility Type'].str.title()

# Clean up the names of the cities
raw_df['City'] = raw_df['City'].replace('Cchicago', 'Chicago')
raw_df['City'] = raw_df['City'].replace('Chcicago', 'Chicago')
raw_df['City'] = raw_df['City'].replace('Chicagoi', 'Chicago')
raw_df['City'] = raw_df['City'].replace('Chicagoo', 'Chicago')
raw_df['City'] = raw_df['City'].replace('Chicagochicago', 'Chicago')
raw_df['City'] = raw_df['City'].replace('Chicagoc', 'Chicago')
raw_df['City'] = raw_df['City'].replace('Chicago.', 'Chicago')
raw_df['City'] = raw_df['City'].replace('Chchicago', 'Chicago')
raw_df['City'] = raw_df['City'].replace('312Chicago', 'Chicago')
raw_df['City'] = raw_df['City'].replace('Chchicago', 'Chicago')
raw_df['City'] = raw_df['City'].replace('Ch', 'Chicago')
raw_df['City'] = raw_df['City'].replace('Chicagobedford Park', 'Bedford Park')
raw_df['City'] = raw_df['City'].replace('Bannockburndeerfield', 'Bannockburn Deerfield')
raw_df['City'] = raw_df['City'].replace('Oolympia Fields', 'Olympia Fields')

# Drop rows where the city is Los Angeles, New York and Inactive
raw_df = raw_df.drop(raw_df[raw_df['City'] == 'Los Angeles'].index)
raw_df = raw_df.drop(raw_df[raw_df['City'] == 'New York'].index)
raw_df = raw_df.drop(raw_df[raw_df['City'] == 'Inactive'].index)

# Drop rows where State isn't IL
raw_df = raw_df.drop(raw_df[raw_df['State'] != 'IL'].index)

# Drop rows where business is not located
raw_df = raw_df.drop(raw_df[raw_df['Results'] == 'Business Not Located'].index)

In [17]:
# Create a dictionary to map the string values to their corresponding integers
risk_mapping = {
    'Risk 1 (High)': 3,
    'Risk 2 (Medium)': 2,
    'Risk 3 (Low)': 1,
    'All': 4
}
# Use the .map() method to replace the string values with integers
raw_df['Risk'] = raw_df['Risk'].map(risk_mapping)

# Handle the case where 'Risk' column might already be transformed or have NaN
raw_df['Risk'] = raw_df['Risk'].fillna(0)
raw_df['Risk'] = raw_df['Risk'].astype(int)

In [18]:
# Define typo corrections
CORRECTIONS = {
    "restuarant": "restaurant"
}

# Define hybrid standardization
HYBRID_STANDARDIZATION = {
    # All variations of Restaurant and Grocery
    r"(restaurant\s*[/&-]?\s*grocery\s*store|grocery\s*store\s*[/&-]?\s*restaurant|restaurant\s*[/&-]?\s*grocery|grocery\s*[/&-]?\s*restaurant)": "Restaurant/Grocery",
    # Restaurant and Liquor
    r"(restaurant\s*[/&-]?\s*liquor|restaurant\s+and\s+liquor)": "Restaurant/Liquor",
}

# Define general category mappings
GENERAL_CATEGORIES = {
    r"(restaurant\s*[/&-]?\s*liquor|restaurant\s+and\s+liquor|tavern\s*[/&-]?\s*restaurant|restaurant\s*\(protein\s+shake\s+bar\))": "Restaurant/Bar",
    r"(alternative\s+school|charter\s+school|private\s+school|school\s+cafeteria|high\s+school\s+kitchen|charter\s+school\s*/\s*cafeteria|teaching\s+school|university\s+cafeteria)": "School",
    r"(daycare.*)": "Daycare",
}

def clean_facility_type(raw_type: str) -> str:
    if pd.isnull(raw_type):
        return raw_type

    # Lowercase and strip
    clean = raw_type.strip().lower()

    # Apply typo corrections
    for typo, correction in CORRECTIONS.items():
        clean = re.sub(rf"\b{typo}\b", correction, clean)

    # Apply hybrid standardizations
    for pattern, replacement in HYBRID_STANDARDIZATION.items():
        if re.search(pattern, clean):
            clean = replacement.lower()

    # Apply general category mappings
    for pattern, replacement in GENERAL_CATEGORIES.items():
        if re.search(pattern, clean):
            clean = replacement.lower()

    # Normalize slashes, ampersands, and whitespace
    clean = re.sub(r"\s*[/&-]\s*", "/", clean)
    clean = re.sub(r"\s+", " ", clean)

    # Title-case for display
    return clean.title()

# Apply the function to your DataFrame
raw_df["Facility Type"] = raw_df["Facility Type"].apply(clean_facility_type)




In [19]:
# Replace some strings
raw_df['Facility Type'] = raw_df['Facility Type'].replace('Shared Kitchen User (Long Term)', 'Shared Kitchen User')

special_event_list = [
    'Banquet Hall', 'Banquets', 'Banquet Hall/Catering', 'Banquet Facility',
    'Catering/Banquet', 'Banquet Dining', 'Banquet Rooms', 'Restaurant.Banquet Halls',
    'Banquet Room', 'Banquet/Kitchen', 'Event Center', 'Event Space', 'Lounge/Banquet Hall',
    'Catered Events', 'Private Event Space', 'Event Venu', 'Catered Events', 'Special Event',
    'Banquet', 'Banquets/Room Service'
]
for alias in special_event_list:
    raw_df['Facility Type'] = raw_df['Facility Type'].replace(alias, 'Special Event Venue')

raw_df['Facility Type'] = raw_df['Facility Type'].replace('Movie Theater', 'Theater')
raw_df['Facility Type'] = raw_df['Facility Type'].replace('Movie Theatre', 'Theater')
raw_df['Facility Type'] = raw_df['Facility Type'].replace('Theatre', 'Theater')


gas_station_list = [
    'Grocery Store/Gas Station', 'Gas Station/Grocery', 'Gas Station/Mini Mart',
    'Gas Station/Restaurant', 'Grocery/Gas Station', 'Gas Station/Store',
    'Grocery/Service Gas Station', 'Gas Station/Food', 'Gas Station/Convenience Store',
    'Gas Station Store', 'Gas Station/Store Grocery', 'Grocery/Service Gas Station',
    'Retail Food/Gas Station', 'Convenience/Gas Station', 'Gas Station/Grocery Store',
    '(Gas Station)', 'Service Gas Station', 'Gas Station/Subway Mini Mart',
    'Grocery(Gas Station)', 'Gas/Mini Mart', 'Gas Station Food Store', 'Gas Station/Subway Mini Mart.'
]
for alias in gas_station_list:
    raw_df['Facility Type'] = raw_df['Facility Type'].replace(alias, 'Gas Station')

church_list = [
    'Church Kitchen', 'Church/Special Events', 'Church (Special Events)', 
    'Church/Special Event', 'Church/Day Care', 'Food Pantry/Church', 
    'Church (Food Pantry)', 'Church/After School Program' 
]

for alias in church_list:
    raw_df['Facility Type'] = raw_df['Facility Type'].replace(alias, 'Church')

mobile_food_list = [
    'Mobile Food Preparer', 'Mobile Food Vendor', 'Mobile Prepared Food Vendor', 'Mobile Frozen Desserts Vendor', 
    'Mobile Food Truck', 'Mobile Desserts Vendor', 'Mobile Frozen Desserts Dispenser/Non/Motorized', 'Mobile Frozen Dessert Vendor',
    'Mobile Frozen Dessert Dispenser_Non Motorized.', 'Mobile Push Cart', 'Mobile Frozen Dessert Disp/Non/Motorized',
    'Mobile Food Desserts Vendor', 'Mobile Frozen Desserts Dispenser/Non/Motor', 'Mobile Food Dispenser',
    'Mobile Dessert Vendor', 'Mobile Dessert Cart'
]

for alias in mobile_food_list:
    raw_df['Facility Type'] = raw_df['Facility Type'].replace(alias, 'Mobile Food')

cooking_list = [
    'Pastry School', 'Culinary Arts School', 'Prep Inside School', 'Cooking Class', 'Cooking School',
    'A/Not/For/Profit Chef Training Program'
]

for alias in cooking_list:
    raw_df['Facility Type'] = raw_df['Facility Type'].replace(alias, 'Culinary School')


grocery_list = [
    'Grocery/Bakery', 'Grocery Store/Bakery', 'Grocery', 'Grocery/Dollar Store', 'Grocery/Deli',
    'Grocery Store/Deli', 'Drug Store/Grocery', 'Dollar/Grocery Store', 'Bakery/Grocery', 
    'Grocery Store Plus Wireless Store', 'Grocery Store/Pharmacy', 'Liquor/Grocery', 'Dollar Store With Grocery',
    'Grocery/Liquor', 'Grocery(Sushi Prep)'
]

for alias in grocery_list:
    raw_df['Facility Type'] = raw_df['Facility Type'].replace(alias, 'Grocery Store')

adult_care_list = [
    'Long Term Care Facility', 'Senior Day Care', 'Long/Term Care Facility', 'Long/Term Care', 'Adult Family Care Center',
    'Assisted Living', 'Supportive Living', 'Assisted Living Senior Care', 'Assissted Living', 'Long Term Care',
    
]

for alias in adult_care_list:
    raw_df['Facility Type'] = raw_df['Facility Type'].replace(alias, 'Adult Care')

child_card_list = [
    "Children'S Services Facility", "1023 Childern'S Services Facility",
    'Childrens Services Facility', "1023 Childern'S Service S Facility", "1023 Children'S Services Facility",
    "Childern'S Services Facility", 'Childern Activity Facility', 'After School Program', 'Before And After School Program',
    'Church/After School Program', 'After School Care', "1023/Children'S Services Facility"
]

for alias in child_card_list:
    raw_df['Facility Type'] = raw_df['Facility Type'].replace(alias, 'Children Services Facility')

daycare_list = [
    'Day Care 2/14', 'Day Care 1023', 'Day Care', '1584/Day Care Above 2 Years', 'Day Care Combo (1586)'
]

for alias in daycare_list:
    raw_df['Facility Type'] = raw_df['Facility Type'].replace(alias, 'Daycare')

popup_list = [
    'Pop/Up Establishment Host/Tier Ii', 'Pop/Up Food Establishment User/Tier Ii','Pop/Up Establishment Host/Tier Iii',
    'Pop/Up Food Establishment User/Tier I' 
]

for alias in popup_list:
    raw_df['Facility Type'] = raw_df['Facility Type'].replace(alias, 'Pop Up Establishment')

cafe_list = [
    'Coffee And/Or Drinks', 'Coffee/Tea', 'Coffee Kiosk', 'Coffee Cart', 'Coffee', 'Riverwalk Cafe', 'Cafe', 'Kids Cafe''',
    'Kids Cafe', 'Coffee Shop', 'Internet Cafe'
]
for alias in cafe_list:
    raw_df['Facility Type'] = raw_df['Facility Type'].replace(alias, 'Cafe')

In [20]:
hybrid_list = [
    'Grocery And Butcher', 'Restaurant And Bar', 'Juice And Salad Bar', 'Exercise And Nutrition Bar',
    'Art Gallery W/Wine And Beer', 'Catering And Wholesale', 'Restaurant/Grocery', 'Restaurant/Bar', 
    'Grocery Store/Cooking School', 'Slaughter House/Grocery', 'Candy/Gelato', 'Grocery Store/Taqueria', 'Catering/Cafe', 
    'Bakery/Deli', 'Convenience/Drug Store', 'Restaurant/Hospital', 'Grocery/Taqueria', 
    'Rest/Grocery', 'Grocery/Butcher', 'Rest/Rooftop', 'Liquor/Grocery Store/Bar', 'Rest/Gym', 'Drug/Food Store', 'Restaurant/Bakery', 
    'Grocery/Drug Store', 'Not/For/Profit Club', 'Herbalife/Zumba', 'Grocery/Cafe', 'Liquor/Coffee Kiosk', 'Bar/Grill', 
    'Grocery/Liquor Store', 'Theater/Bar', 'Bakery/Restaurant',  'Cafe/Store', 'Bowling Lanes/Banquets', 
    'Beverage/Silverware Warehouse', 'Tavern/Bar', 'Test Kitchen/Storage', 'Deli/Bakery', 'Juice Bar/Grocery', 
    'Paleteria/Icecream Shop', 'Art Gallery W/Wine And Beer', 'Nutrition/Herbalife', 'Liquore Store/Bar', 'Museum/Gallery', 
    'Restaurant/Bar/Theater', 'Tavern/Packaged Goods', 'Tavern/Liquor', 'Retail Wine/Wine Bar', 
    'Wholesale/Retail', 'Shakes/Teas', 'Drug Store/W/Food', 'Tavern/1006', 'Frozen Desserts Dispenser/Non Motorized', 
    'Frozen Desserts Dispenser/Non/Motorized', 'Health Center/Nutrition Classes', 'Cat/Liquor', 'Np/Kiosk', 'Tap Room/Tavern/Liquor Store', 
    'Service Bar/Theatre'
]

for alias in hybrid_list:
    raw_df['Facility Type'] = raw_df['Facility Type'].replace(alias, 'Hybrid')

In [21]:
print(raw_df['Facility Type'].unique())

['Restaurant' 'Grocery Store' 'Liquor' 'School' 'Catering' 'Stadium'
 'Daycare' 'Convenient Store' 'Nan' 'Children Services Facility'
 'Adult Care' 'Bakery' 'Mobile Food' 'Shared Kitchen User'
 'Shared Kitchen' 'Special Event Venue' 'Kitchen Demo' 'Gas Station'
 'Tavern' 'Culinary School' 'Nursing Home' 'Live Poultry' 'Donut Shop'
 'Hybrid' 'Hospital' 'Golden Diner' 'Paleteria' 'Shelter' 'Youth Housing'
 'Cafeteria' 'Juice Bar' 'Rooftop' 'Rooftops' 'Hotel' 'Navy Pier Kiosk'
 'Herbalife' 'Live Poultry Slaughter Facility' 'Theater' 'Roof Tops'
 'Food Pantry' 'Ice Cream Shop' 'Roof Top' 'Regulated Business'
 'Packaged Health Foods' 'Commissary' 'Milk Tea' 'Herbal Drinks'
 'Retail Store Offers Cooking Classes' 'Cafe' 'Pop Up Establishment'
 'Dining Hall' 'Public Shcool' 'Wholesale' 'College' 'Butcher, Deli'
 'Unlicensed Facility' 'Hostel' 'Church' 'Bar' 'Convenience Store'
 'Riverwalk' 'Other' 'Archdiocese' 'Poultry Slaughter'
 'Thc Infused Bakery' 'Entertainment Venue' 'Music Venue' 'News

In [22]:
inspection_value_count_series = raw_df['Inspection Type'].value_counts()
insp_type_value_counts_df = inspection_value_count_series.reset_index()
insp_type_value_counts_df.columns = ['unique_value', 'count']
insp_type_value_counts_df.to_csv('inspection_type.csv')

In [23]:
facility_type_value_count_series = raw_df['Facility Type'].value_counts()
facility_type_value_counts_df = facility_type_value_count_series.reset_index()
facility_type_value_counts_df.columns = ['unique_value', 'count']

# Filter for Facility Types that have at least 50 values 
high_count_facility_types_df = facility_type_value_counts_df[facility_type_value_counts_df['count'] >= 50]
high_count_facility_types_list = high_count_facility_types_df['unique_value'].tolist()
filtered_df = raw_df[raw_df['Facility Type'].isin(high_count_facility_types_list)].copy() # Subset from raw_df

# Create new column to indicate if establishment passed
pass_conditions = ['pass', 'pass w/ conditions', 'pass with conditions']
filtered_df['IsPass'] = filtered_df['Results'].str.strip().str.lower().isin(pass_conditions)

print(filtered_df.shape)

(288414, 18)


In [24]:
facility_type_value_count_series = filtered_df['Facility Type'].value_counts()
facility_type_value_counts_df = facility_type_value_count_series.reset_index()
facility_type_value_counts_df.columns = ['unique_value', 'count']
facility_type_value_counts_df.to_csv('facility_type.csv')

In [25]:
filtered_df.to_csv('../Datasets/Clean_Food_Inspections.csv')

In [26]:
from shapely.geometry import shape
import ast
import geopandas as gpd

crimes = pd.read_csv("../Datasets/crimes_last2years.csv")
df_boundaries = pd.read_csv("../Datasets/community_boundaries.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../Datasets/crimes_last2years.csv'

In [ ]:
parsed_geometries = [shape(ast.literal_eval(g)) for g in df_boundaries["the_geom"]]
df_boundaries["geometry"] = parsed_geometries
gdf_boundaries = gpd.GeoDataFrame(df_boundaries, geometry="geometry", crs="EPSG:4326")

In [ ]:
gdf_df = gpd.GeoDataFrame(filtered_df, geometry=gpd.points_from_xy(filtered_df["Longitude"], filtered_df["Latitude"]), crs="EPSG:4326") 
gdf_df = gpd.sjoin(gdf_df, gdf_boundaries[["community", "area_numbe", "geometry"]], how="left", predicate="within").drop(columns=["index_right"]).rename(columns={"community": "community", "area_numbe": "area_number"})

In [ ]:
gdf_df

,Inspection ID,DBA Name,AKA Name,License Number,Facility Type,Risk,Address,City,State,Zip,...,Inspection Type,Results,Violations,Latitude,Longitude,Location,IsPass,geometry,community,area_number
0,1170324,CAPTAIN HOOKS FISH & CHICKEN/FIREHOUSE GRILL,CAPTAIN HOOKS FISH & CHICKEN FIREHOUSE GRILL,1804138.0,Restaurant,3,5648 W ROOSEVELT,Chicago,IL,60644.0,...,Complaint,Pass w/ Conditions,21. * CERTIFIED FOOD MANAGER ON SITE WHEN POTE...,41.865679,-87.766331,"(41.86567851033532, -87.76633113301574)",True,POINT (-87.76633 41.86568),AUSTIN,25.0
1,2608378,Babas Halal,Babas Halal,2684170.0,Restaurant,3,7901 S DAMEN AVE,Chicago,IL,60620.0,...,Complaint,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.750189,-87.672986,"(41.750189342293375, -87.67298583977204)",False,POINT (-87.67299 41.75019),AUBURN GRESHAM,71.0
2,2614750,CHICK-FIL-A PULLMAN,CHICK-FIL-A PULLMAN,3015728.0,Restaurant,3,11131 S CORLISS AVE,Chicago,IL,60628.0,...,License,Pass,58. ALLERGEN TRAINING AS REQUIRED - Comments: ...,41.691956,-87.600558,"(41.69195600663339, -87.60055824712973)",True,POINT (-87.60056 41.69196),PULLMAN,50.0
3,2609909,HAPPY MARKET,HAPPY MARKET,2912802.0,Grocery Store,2,2334 S WENTWORTH AVE,Chicago,IL,60616.0,...,Canvass,Pass w/ Conditions,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.849954,-87.632094,"(41.84995400192252, -87.63209419559098)",True,POINT (-87.63209 41.84995),ARMOUR SQUARE,34.0
4,2609927,SAT KAIVAL FOOD INC/SUBWAY,SAT KAIVAL FOOD INC/SUBWAY,2728400.0,Restaurant,3,1916 S STATE ST,Chicago,IL,60616.0,...,Canvass,Pass,36. THERMOMETERS PROVIDED & ACCURATE - Comment...,41.856053,-87.627311,"(41.85605269621059, -87.62731125804903)",True,POINT (-87.62731 41.85605),NEAR SOUTH SIDE,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289845,2610937,DAPPER'S EAST RESTAURANT,DAPPER'S EAST RESTAURANT,12540.0,Restaurant,3,2901 W ADDISON ST,Chicago,IL,60618.0,...,Complaint Re-Inspection,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,41.946553,-87.700536,"(41.94655282616904, -87.700535765616)",True,POINT (-87.70054 41.94655),AVONDALE,21.0
289846,2610845,SHARPIE'S,SHARPIE'S,3002203.0,Restaurant,1,1721 N ELSTON AVE,Chicago,IL,60642.0,...,License,Not Ready,NaN,41.913729,-87.664223,"(41.91372879440366, -87.66422290709808)",False,POINT (-87.66422 41.91373),WEST TOWN,24.0
289847,2610409,ACE SUSHI @POTASH MARKET,ACE SUSHI,2808766.0,Restaurant,3,875 N STATE ST,Chicago,IL,60610.0,...,Canvass,Pass,"55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ...",41.898878,-87.628111,"(41.89887819742381, -87.6281111483033)",True,POINT (-87.62811 41.89888),NEAR NORTH SIDE,8.0
289848,2610823,"Kitchen Chicago, LLC","Kitchen Chicago, LLC",2119905.0,Shared Kitchen,3,324 N LEAVITT ST,Chicago,IL,60612.0,...,Canvass,Pass,49. NON-FOOD/FOOD CONTACT SURFACES CLEAN - Com...,41.887434,-87.681849,"(41.88743405025222, -87.68184949426895)",True,POINT (-87.68185 41.88743),NEAR WEST SIDE,28.0


In [ ]:
gdf_crimes = gpd.GeoDataFrame(crimes, geometry=gpd.points_from_xy(crimes["longitude"].astype(float), crimes["latitude"].astype(float)), crs="EPSG:4326")
gdf_crimes = gpd.sjoin(
    gdf_crimes,
    gdf_boundaries[["community", "area_numbe", "geometry"]],
    how="left",
    predicate="within"
).drop(columns=["index_right"]).rename(columns={"community": "community", "area_numbe": "area_number"})


In [ ]:
crime_summary = gdf_crimes.groupby("area_number").agg(
    total_crimes=("id", "count"),
    arrest_rate=("arrest", lambda x: x.mean() if x.dtype == bool else pd.to_numeric(x).mean())
    ).reset_index()

crime_type_counts = (
    gdf_crimes
    .groupby(["area_number", "primary_type"])
    .size()
    .unstack(fill_value=0)
    .add_prefix("count_")
    .reset_index()
)

crime_summary = pd.merge(crime_summary, crime_type_counts, on="area_number", how="left")
crime_summary.columns = [col.lower() for col in crime_summary.columns]


In [ ]:
acs_df = pd.read_csv("../Datasets/acs_by_community.csv")
acs_df = acs_df.drop(columns=['acs_year', 'record_id']).rename(columns={'community_area': 'community'})

In [ ]:
main_df = pd.merge(gdf_df, crime_summary, on="area_number", how="left")
main_df = pd.merge(main_df, acs_df, on="community", how="left")
main_df

,Inspection ID,DBA Name,AKA Name,License Number,Facility Type,Risk,Address,City,State,Zip,...,total_population,white,black_or_african_american,american_indian_or_alaska,asian,native_hawaiin_or_pacific,other_race,multiracial,white_not_hispanic_or_latino,hispanic_or_latino
0,1170324,CAPTAIN HOOKS FISH & CHICKEN/FIREHOUSE GRILL,CAPTAIN HOOKS FISH & CHICKEN FIREHOUSE GRILL,1804138.0,Restaurant,3,5648 W ROOSEVELT,Chicago,IL,60644.0,...,100278.0,10447.0,73602.0,531.0,678.0,21.0,8512.0,6487.0,5386.0,19591.0
1,2608378,Babas Halal,Babas Halal,2684170.0,Restaurant,3,7901 S DAMEN AVE,Chicago,IL,60620.0,...,46483.0,760.0,43414.0,119.0,399.0,0.0,993.0,798.0,491.0,1577.0
2,2614750,CHICK-FIL-A PULLMAN,CHICK-FIL-A PULLMAN,3015728.0,Restaurant,3,11131 S CORLISS AVE,Chicago,IL,60628.0,...,6851.0,1123.0,5293.0,0.0,5.0,0.0,244.0,186.0,848.0,559.0
3,2609909,HAPPY MARKET,HAPPY MARKET,2912802.0,Grocery Store,2,2334 S WENTWORTH AVE,Chicago,IL,60616.0,...,13149.0,2556.0,1487.0,107.0,8402.0,61.0,212.0,325.0,2226.0,565.0
4,2609927,SAT KAIVAL FOOD INC/SUBWAY,SAT KAIVAL FOOD INC/SUBWAY,2728400.0,Restaurant,3,1916 S STATE ST,Chicago,IL,60616.0,...,26307.0,14113.0,6588.0,57.0,3830.0,0.0,329.0,1390.0,13320.0,1334.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288200,2610937,DAPPER'S EAST RESTAURANT,DAPPER'S EAST RESTAURANT,12540.0,Restaurant,3,2901 W ADDISON ST,Chicago,IL,60618.0,...,36350.0,19289.0,927.0,263.0,1750.0,15.0,7809.0,6297.0,13718.0,18606.0
288201,2610845,SHARPIE'S,SHARPIE'S,3002203.0,Restaurant,1,1721 N ELSTON AVE,Chicago,IL,60642.0,...,86453.0,61759.0,6039.0,147.0,5138.0,21.0,4650.0,8700.0,54505.0,17099.0
288202,2610409,ACE SUSHI @POTASH MARKET,ACE SUSHI,2808766.0,Restaurant,3,875 N STATE ST,Chicago,IL,60610.0,...,91635.0,64903.0,6434.0,492.0,12354.0,8.0,1855.0,5590.0,62702.0,6664.0
288203,2610823,"Kitchen Chicago, LLC","Kitchen Chicago, LLC",2119905.0,Shared Kitchen,3,324 N LEAVITT ST,Chicago,IL,60612.0,...,65370.0,30838.0,16099.0,119.0,11600.0,88.0,2322.0,4304.0,28282.0,6779.0
